In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('/sep/haipeng/FWI/FiberFWI/toolbox/')
sys.path.append('../src/')

import numpy as np
from scipy.interpolate import interp2d
import matplotlib.pyplot as plt

from simulate import Simulate
from utils import wavelet

from plot_tools import plot_wavefield, plot_model

In [3]:
# Function to interpolate a 70x70 array to 128x64
def interpolate_array(input_array, nx, nz):
    if input_array.shape != (70, 70):
        raise ValueError("Input array must be of shape (70, 70)")

    # Define the original grid points
    x = np.linspace(0, 1, 70)
    y = np.linspace(0, 1, 70)
    
    # Create an interpolation function
    f = interp2d(x, y, input_array.T, kind='cubic')
    
    # Define the new grid points where you want to interpolate
    x_new = np.linspace(0, 1, nx)
    y_new = np.linspace(0, 1, nz)
    
    # Perform the interpolation
    result = f(x_new, y_new)
    
    return result.T

## Load different geological models

### Homogenerous models

In [4]:
vp_homo = []
vs_homo = []
rho_homo = []

nx = 128
nz = 32

for i in range(500):
    # generate true model by increasing vp linearly
    vp  = np.ones((nx, nz), dtype=np.float32) * (3000 + 3 * i)
    vs  = vp / 1.732
    rho = np.power(vp, 0.25) * 310

    vp_homo.append(vp)
    vs_homo.append(vs)
    rho_homo.append(rho)

vp_homo = np.array(vp_homo)
vs_homo = np.array(vs_homo)
rho_homo = np.array(rho_homo)

In [5]:
vp_model = vp_homo
vs_model = vs_homo
rho_model = rho_homo

In [6]:
vp_model.shape

(500, 128, 32)

In [7]:
# # Use plot_wavefield in Jupyter notebook
# from IPython.display import HTML

# wavefield_ani = plot_model(vp_model[::10,:,:])
# HTML(wavefield_ani.to_jshtml())

In [8]:
# System
path    = '/scr3/haipeng/fno-fd3domp/homo_data'
cluster = 'local'  # 'slurm'
taskNum = 1
thdNum  = 4
comprR  = 1E-4

# Model
dim = '2D'
nx, ny, nz = [128,     1,  32]
dx, dt, nt = [ 10, 0.001, 801]
fs, pml    = [True,        20]

# Source
domFreq   = 15.
srcNum    = 1
srcType   = 'vector'
amp       = 1e7
srcWvlt   = [wavelet(nt, dt, domFreq, 'Ricker') *amp] * srcNum
vectorAngle= [np.array([ 0, 90])]* srcNum
srcCoord = [np.array([640, 0, dx]) for i in range(srcNum)]

# Receiver
rcvComp = ['vx', 'vz', 'sr', 'pr']
rcvNum = nx
rcvCoord = np.zeros((rcvNum, 3))
for i in range(rcvNum):
    rcvCoord[i, :] = [dx * i, 0.0, dx * 2]
rcvCoord = [rcvCoord] * srcNum


In [9]:
vp_all = []
vs_all = []
rho_all = []
vx_all = []
vz_all = []

nsample = vp_model.shape[0]

for i in range(nsample):
    
    print(f"{i+1} / {nsample}")
    
    vp  = vp_model[i].reshape((nx, ny, nz))
    vs  = vs_model[i].reshape((nx, ny, nz))
    rho = rho_model[i].reshape((nx, ny, nz))

    if np.any(vp / (vs + 0.001) < np.sqrt(2)):
        vs = vp /1.732
        
    trueModel = {'vp': vp, 'vs': vs, 'rho': rho}
    
    # clean previous data
    !rm -r /scr3/haipeng/fno-fd3domp/homo_data/data/obs/*
    
    # Simulation
    simulate = Simulate(path=path, cluster=cluster, taskNum=taskNum, thdNum=thdNum, comprR=comprR,
                      dim=dim, nx=nx, ny=ny, nz=nz, dx=dx, dt=dt, nt=nt, fs=fs, pml=pml, model=trueModel,
                      srcNum=srcNum, srcType=srcType, domFreq=domFreq, srcCoord=srcCoord, srcWvlt=srcWvlt,
                        vectorAngle = vectorAngle, rcvCoord=rcvCoord, rcvComp=rcvComp)
    
    
    # set time interval to be 2
    simulate.saveStep = 1
    
    ## run simulation
    simulate.forward(simuType='obs', saveSnap = True, snapComp = ['vx', 'vz'])

    # load vx data
    wavefields_vx = []
    for i in range(nt):
        wavefield = simulate.loadWavefield(simuType='obs', isrc = 0, isnap=i, comp = 'vx')
        wavefields_vx.append(wavefield.T)
        
    wavefields_vz = []
    for i in range(nt):
        wavefield = simulate.loadWavefield(simuType='obs', isrc = 0, isnap=i, comp = 'vz')
        wavefields_vz.append(wavefield.T)
    
    wavefields_vx = np.array(wavefields_vx)
    wavefields_vz = np.array(wavefields_vz)

    vp_all.append(vp)
    vs_all.append(vs)
    rho_all.append(rho)
    vx_all.append(wavefields_vx)
    vz_all.append(wavefields_vz)


1 / 500
rm: cannot remove '/scr3/haipeng/fno-fd3domp/homo_data/data/obs/*': No such file or directory
*****************************************************

      Seismic Waveform Inversion Toolbox          

*****************************************************

Forward modeling: 2-D Cartesian Geometry
Forward modeling: nx = 128, nz = 32, dx = 10.0 m
Forward modeling: x  = 0 ~ 1.27 km
Forward modeling: z  = 0 ~ 0.31 km
Forward modeling: nt = 801, dt = 1.00 ms, time = 0.8 s
Forward modeling: vp = 3000.0 ~ 3000.0 m/s
Forward modeling: vs = 1732.1 ~ 1732.1 m/s
Forward modeling: Sources number = 1, vector
Forward modeling: Receiver component: ['vx', 'vz', 'sr', 'pr']
Forward modeling: Parallel: 1 task, 4 thread(s) per source

2 / 500
*****************************************************

      Seismic Waveform Inversion Toolbox          

*****************************************************

Forward modeling: 2-D Cartesian Geometry
Forward modeling: nx = 128, nz = 32, dx = 10.0 m
Forward

In [10]:
# convert to numpy arrays
vp_all = np.array(np.squeeze(vp_all))
vs_all = np.array(np.squeeze(vs_all))
rho_all = np.array(np.squeeze(rho_all))
vx_all = np.array(vx_all)
vz_all = np.array(vz_all)

In [11]:
print(vp_all.shape)
print(vs_all.shape)
print(rho_all.shape)
print(vx_all.shape)
print(vz_all.shape)

(500, 128, 32)
(500, 128, 32)
(500, 128, 32)
(500, 801, 128, 32)
(500, 801, 128, 32)


In [12]:
ns, nt, nx, nz = vx_all.shape
print(ns, nt, nx, nz)

500 801 128 32


In [13]:
np.save(f'/scr2/haipeng/openfwi_dataset_train/vp_homo_ns{ns}_nx{nx}_nz{nz}.npy', vp_all)
np.save(f'/scr2/haipeng/openfwi_dataset_train/vs_homo_ns{ns}_nx{nx}_nz{nz}.npy', vs_all)
np.save(f'/scr2/haipeng/openfwi_dataset_train/rho_homo_ns{ns}_nx{nx}_nz{nz}.npy', rho_all)
np.save(f'/scr2/haipeng/openfwi_dataset_train/vx_homo_ns{ns}_nt{nt}_nx{nx}_nz{nz}.npy', vx_all)
np.save(f'/scr2/haipeng/openfwi_dataset_train/vz_homo_ns{ns}_nt{nt}_nx{nx}_nz{nz}.npy', vz_all)

In [14]:
print(f'/scr2/haipeng/openfwi_dataset_train/vp_homo_ns{ns}_nx{nx}_nz{nz}.npy')

/scr2/haipeng/openfwi_dataset_train/vp_homo_ns500_nx128_nz32.npy


In [15]:
# Use plot_wavefield in Jupyter notebook
from IPython.display import HTML

wavefield_ani = plot_wavefield(vx_all[0,::10,:,:])
HTML(wavefield_ani.to_jshtml())